In [76]:
import ROOT
import numpy as np

%jsroot on

In [ ]:
def tof_1mus_to_En(TOF: any, L: float) -> any:
    """Converts TOF in mu s to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L) / (TOF))**2

def tof_10ns_to_En(TOF: any, L: float) -> any:
    """Converts TOF in units of 10 ns to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L) / (TOF/100))**2

def tof_1ns_to_En(TOF: any, L: float) -> any:
    """Converts TOF in units of ns to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L) / (TOF*10))**2

In [137]:
def get_cross_section(file_name: str, xbins: list[float]) -> ROOT.TH1D:
    """Take a file name as a string and returns the cross section """

    file = open(file_name, "r")
    file.readline()
    
    energy = []
    crossSection = []

    for line in file:
        values = line.strip().split(',')
        energy.append(float(values[0]))  # eV
        crossSection.append(float(values[1]))  # barns

    file.close()


    graph = ROOT.TGraph(len(energy), np.array(energy), np.array(crossSection))

    hEn_sig_tot_10Bo = ROOT.TH1D("hEn_sig_tot_10Bo", "hEn_sig_tot_10Bo; Energy [eV]; Cross Section [b]", len(xbins)-1, xbins)

    for i in range(1, len(xbins)):
        x = hEn_sig_tot_10Bo.GetBinCenter(i)
        y = graph.Eval(x)
        hEn_sig_tot_10Bo.SetBinContent(i, y)

    return hEn_sig_tot_10Bo

    
def get_xbins_En(down: int, up: int, Nbins: int) -> np.array(float):
    """Take the lower limit, upper limit, and Nbins of the TOF histogram and returns an xbins of En"""

    xbins_tof = np.linspace(up/Nbins, up+up/Nbins, Nbins+1)
    xbins_En=tof_mus_to_En(xbins_tof,21.5)
    
    return np.sort(xbins_En)


def get_beam_intensity(file_name: str, cross_section: ROOT.TH1D) -> ROOT.TH1D:
    """Test"""
    
    file_in = ROOT.TFile(file_name, "READ")

    hEn_all_gated_Bo = file_in.Get("hEn_all_gated_Bo")

    beam_intensity = hEn_all_gated_Bo

    print(cross_section.GetNbinsX())
    print(beam_intensity.GetNbinsX())
    
    # beam_intensity.SetName("beam_intensity")
    # beam_intensity.GetYaxis().SetName("")
    beam_intensity.Divide(cross_section)

    return beam_intensity


In [138]:
Nbins = 10000
down = 0
up =100

xbins_En = get_xbins_En(down, up, Nbins)

In [129]:
cross_section = get_cross_section("../data/nndc_data.txt", xbins_En)
cross_section.Draw()
ROOT.gROOT.GetListOfCanvases().Draw()

Warning in <TROOT::Append>: Replacing existing TH1: hEn_sig_tot_10Bo (Potential memory leak).


In [139]:
beam_intensity = get_beam_intensity("../data/stage0_output_beam_23_7_5.root", cross_section)
beam_intensity.Draw()
ROOT.gROOT.GetListOfCanvases().Draw()

10000
10000


AttributeError: 'CPyCppyy_NoneType' object has no attribute 'Draw'

Warning in <TH1D::Divide>: Dividing histograms with different axis limits
